# 🧮 Roll Rate Model (Markov Chain)
### Demo Notebook - Offline Mode (Parquet, No Oracle)
This notebook loads a parquet dataset, builds transition matrices, runs 12‑month forecast and performs backtesting.

In [1]:
# ============================================================
# ✅ FIX PATH FOR NOTEBOOK INSIDE /notebooks
# ============================================================
import sys, os
from pathlib import Path

# Xác định thư mục cha của notebooks/
project_root = Path(os.getcwd()).parent  # => C:\Users\User\RR_model
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))  # ⚠️ thêm project_root, KHÔNG thêm src

print("📁 Project root added:", project_root)

# Kiểm tra lại
import importlib.util
print("🔍 src module found:", importlib.util.find_spec("src"))


📁 Project root added: D:\Python_code\RR_Model
🔍 src module found: ModuleSpec(name='src', loader=<_frozen_importlib_external.SourceFileLoader object at 0x000001CFE68A3880>, origin='D:\\Python_code\\RR_Model\\src\\__init__.py', submodule_search_locations=['D:\\Python_code\\RR_Model\\src'])


In [8]:
import numpy as np


In [2]:


# Sau đó import như bình thường:
from src.config import CFG, BUCKETS_CANON, OUT_ROOT, PARQUET_DIR, PARQUET_FILE
from src.rollrate.transition import compute_transition
from src.rollrate.forecast import forecast_report
from src.rollrate.transition import compute_transition_by_mob
from collections import defaultdict
from src.rollrate.backtest import (
    matrix_stability_score,
    rollforward_validation,
    plot_matrix_diff,
    plot_distribution_compare,
)
from src.data_loader import load_data

print("✅ Import thành công.")


✅ Import thành công.


In [3]:
# Load parquet dataset
df = load_data(r"D:\Python_code\RR_Model\sql\NTB.sql")
import pandas as pd
from pandas.tseries.offsets import MonthEnd

# Đảm bảo DISBURSAL_DATE đúng định dạng datetime
df["DISBURSAL_DATE"] = pd.to_datetime(df["DISBURSAL_DATE"], format="%m/%d/%Y %H:%M", errors="coerce")
df["DISBURSAL_DATE"] = df["DISBURSAL_DATE"] + MonthEnd(0)
df["MOB"] = df["MOB"].fillna(0).astype(int)

# ✅ Tính cutoff cho từng dòng
df["CUTOFF_DATE"] = df.apply(lambda x: x["DISBURSAL_DATE"] + MonthEnd(x["MOB"]), axis=1)

print("✅ Added CUTOFF_DATE from DISBURSAL_DATE + MOB months")
print(df[["DISBURSAL_DATE", "MOB", "CUTOFF_DATE"]].head(10))


df.head()

🔗 Loading data from Oracle...
=== SQL DEBUG ===
File: D:\Python_code\RR_Model\sql\NTB.sql
First 200 chars:
 select  CUTOFF_DATE, AGREEMENT_ID, DISBURSAL_DATE, DISBURSAL_AMOUNT, DPD_EOM, RISK_BUCKET, PRINCIPLE_OUTSTANDING, STATUS, MOB, MAFC_SUB_CATEGORY PRODUCT_TYPE, NPA_STAGEID, DPD_GROUP, STATE_MODEL, CASE...
Params: {}


D:\Python_code\RR_Model\src\db.py:63: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(final_sql, conn, params=bind_params)


✅ Added CUTOFF_DATE from DISBURSAL_DATE + MOB months
  DISBURSAL_DATE  MOB CUTOFF_DATE
0     2023-01-31    0  2023-01-31
1     2023-01-31    1  2023-02-28
2     2023-01-31    2  2023-03-31
3     2023-01-31    3  2023-04-30
4     2023-01-31    4  2023-05-31
5     2023-01-31    5  2023-06-30
6     2023-01-31    6  2023-07-31
7     2023-01-31    7  2023-08-31
8     2023-01-31    8  2023-09-30
9     2023-01-31    9  2023-10-31


,CUTOFF_DATE,AGREEMENT_ID,DISBURSAL_DATE,DISBURSAL_AMOUNT,DPD_EOM,RISK_BUCKET,PRINCIPLE_OUTSTANDING,STATUS,MOB,PRODUCT_TYPE,NPA_STAGEID,DPD_GROUP,STATE_MODEL,RISK_SCORE
0,2023-01-31,4089415,2023-01-31,18309000.0,0,B0,18309000,A,0,SALPIL,REGULAR,DPD0,DPD0,B
1,2023-02-28,4089415,2023-01-31,18309000.0,0,B0,18273533,A,1,SALPIL,REGULAR,DPD0,DPD0,B
2,2023-03-31,4089415,2023-01-31,18309000.0,0,B0,17959922,A,2,SALPIL,REGULAR,DPD0,DPD0,B
3,2023-04-30,4089415,2023-01-31,18309000.0,0,B0,17703148,A,3,SALPIL,REGULAR,DPD0,DPD0,B
4,2023-05-31,4089415,2023-01-31,18309000.0,0,B0,17414031,A,4,SALPIL,REGULAR,DPD0,DPD0,B


In [ ]:
# Compute simple per-product transitions from the whole dataset
matrices = {}
for subprod in df['PRODUCT_TYPE'].dropna().unique():
    sub_df = df[df['PRODUCT_TYPE'] == subprod].copy()
    sub_df['COUNT'] = 1
    matrices[subprod] = compute_transition(sub_df, value_col='COUNT')
list(matrices.keys())

In [ ]:
from src.rollrate.transition import compute_transition_by_mob

matrices_by_mob = compute_transition_by_mob(df)

for seg, mob_dict in matrices_by_mob.items():
    print(f"\nSegment {seg}: {len(mob_dict)} MOB matrices")
    print("MOBs:", list(mob_dict.keys())[:10])


In [ ]:
# Forecast 12 months using the latest snapshot
latest_month = df[CFG['cutoff']].max()
df_latest = df[df[CFG['cutoff']] == latest_month].copy()
reports, summary = forecast_report(df_latest, matrices, months=12, value_col=CFG['ead'])
summary

In [ ]:
# Backtest: build a noisy matrix and compare
P = list(matrices.values())[0]
P2 = (P * (1 + np.random.normal(0, 0.05, P.shape))).clip(lower=0)
P2 = P2.div(P2.sum(axis=1), axis=0)
score = matrix_stability_score(P, P2)
print(f'Matrix Stability Score: {score:.4f}')
plot_matrix_diff(P, P2, title='Δ Transition Matrix')

In [ ]:
# Sau khi đã có df["CUTOFF_DATE"]
df["CUTOFF_DATE_STR"] = df["CUTOFF_DATE"].dt.strftime("%Y%m")

# Rồi cập nhật CFG tạm thời
CFG["cutoff"] = "CUTOFF_DATE_STR"

# Giờ gọi lại backtest
start_month = str(sorted(df[CFG['cutoff']].unique())[0])
rf = rollforward_validation(df, P, start_month=start_month, horizon=2)


In [ ]:
# Roll-forward validation from the first available month
start_month = str(sorted(df[CFG['cutoff']].unique())[0])
rf = rollforward_validation(df, P, start_month=start_month, horizon=2)
plot_distribution_compare(rf, title='Roll-forward (Predicted vs Actual)')

In [ ]:
# Save summary
OUT_ROOT.mkdir(parents=True, exist_ok=True)
summary.to_excel(OUT_ROOT / 'forecast_summary_demo.xlsx', index=False)
print(f'✅ Demo completed | Output saved at: {OUT_ROOT}')

In [ ]:
print(f"Số bảng transition hiện có: {len(matrices_by_mob)}")
print("Danh sách các segment/product:")
for name in matrices_by_mob.keys():
    print(" -", name)


In [ ]:
from src.rollrate.transition import compute_transition_by_mob

matrices_by_mob = compute_transition_by_mob(df)

for seg, mob_dict in matrices_by_mob.items():
    print(f"\nSegment {seg}: {len(mob_dict)} MOB matrices")
    print("MOBs:", list(mob_dict.keys())[:10])


In [ ]:
for seg, mob_dict in matrices_by_mob.items():
    print(f"\nSegment {seg}: {len(mob_dict)} MOB matrices")
    print("MOB list:", list(mob_dict.keys()))


In [ ]:
mat = matrices_by_mob["A"][1]
print(mat.round(3))


In [ ]:
print("Row sums:", mat.sum(axis=1).round(3))


In [ ]:
import pandas as pd
import numpy as np

# 1) Cột state & ead có tồn tại/đúng tên?
print("state col =", CFG["state"], "| in df? ->", CFG["state"] in df.columns)
print("ead   col =", CFG["ead"],   "| in df? ->", CFG["ead"]   in df.columns)

# 2) Phân bố state hiện tại
print(df[CFG["state"]].value_counts().head(10))

# 3) Tổng EAD > 0 ?
if CFG["ead"] in df.columns:
    print("Total EAD =", df[CFG["ead"]].fillna(0).sum())


In [ ]:
# đảm bảo có thứ tự thời gian
time_col = "MOB" if "MOB" in df.columns else CFG.get("cutoff", None)
assert time_col is not None, "Cần MOB hoặc CUTOFF_DATE để tạo cặp chuyển trạng thái."

# sort và tạo state ở t và t+1 theo từng loan
g = df.sort_values([CFG["loan"], time_col]).groupby(CFG["loan"], group_keys=False)

pairs = []
# state_t (hiện tại), state_t1 (tháng sau), weight (EAD hoặc 1)
tmp = pd.DataFrame({
    "state_t":  g[CFG["state"]].shift(0),
    "state_t1": g[CFG["state"]].shift(-1),
    "weight":   (df[CFG["ead"]] if CFG["ead"] in df.columns else 1.0)
})
# cần các cột key để ghép đúng
tmp[CFG["loan"]] = df[CFG["loan"]].values
tmp[time_col]    = df[time_col].values

# chỉ lấy record có cả t và t+1 trong cùng khoản vay
tmp = tmp.dropna(subset=["state_t","state_t1"])
print("pairs rows:", len(tmp))
tmp.head()


In [10]:
df = load_data(r"D:\Python_code\RR_Model\sql\NTB.sql")

🔗 Loading data from Oracle...
=== SQL DEBUG ===
File: D:\Python_code\RR_Model\sql\NTB.sql
First 200 chars:
 select  CUTOFF_DATE, AGREEMENT_ID, DISBURSAL_DATE, DISBURSAL_AMOUNT, DPD_EOM, RISK_BUCKET, PRINCIPLE_OUTSTANDING, STATUS, MOB, MAFC_SUB_CATEGORY PRODUCT_TYPE, NPA_STAGEID, DPD_GROUP, STATE_MODEL, CASE...
Params: {}


D:\Python_code\RR_Model\src\db.py:63: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(final_sql, conn, params=bind_params)


In [11]:
PARQUET_DIR = Path("data")  # cùng đường dẫn mà hàm load dùng
PARQUET_DIR.mkdir(exist_ok=True)

df.to_parquet(PARQUET_DIR / "my_data.parquet", compression="gzip", index=False)

ImportError: Unable to find a usable engine; tried using: 'pyarrow', 'fastparquet'.
A suitable version of pyarrow or fastparquet is required for parquet support.
Trying to import the above resulted in these errors:
 - Missing optional dependency 'pyarrow'. pyarrow is required for parquet support. Use pip or conda to install pyarrow.
 - Missing optional dependency 'fastparquet'. fastparquet is required for parquet support. Use pip or conda to install fastparquet.

ArrowKeyError: No type extension with name arrow.py_extension_type found

In [13]:
pip install pyarrow



   ---------------------------------------- 0.0/28.1 MB ? eta -:--:--
   ---------------------------------------- 0.3/28.1 MB ? eta -:--:--
   - -------------------------------------- 1.3/28.1 MB 4.8 MB/s eta 0:00:06
   --- ------------------------------------ 2.4/28.1 MB 5.0 MB/s eta 0:00:06
   ---- ----------------------------------- 2.9/28.1 MB 4.7 MB/s eta 0:00:06
   ------ --------------------------------- 4.7/28.1 MB 5.5 MB/s eta 0:00:05
   ------- -------------------------------- 5.2/28.1 MB 5.1 MB/s eta 0:00:05
   --------- ------------------------------ 6.8/28.1 MB 5.4 MB/s eta 0:00:04
   ------------- -------------------------- 9.2/28.1 MB 6.1 MB/s eta 0:00:04
   --------------- ------------------------ 11.0/28.1 MB 6.5 MB/s eta 0:00:03
   ------------------ --------------------- 13.1/28.1 MB 7.0 MB/s eta 0:00:03
   --------------------- ------------------ 14.9/28.1 MB 7.2 MB/s eta 0:00:02
   ----------------------- ---------------- 16.8/28.1 MB 7.3 MB/s eta 0:00:02
   -----

In [4]:
import pandas as pd
from pathlib import Path

PARQUET_DIR = Path("data")
PARQUET_DIR.mkdir(exist_ok=True)

# Ghi DataFrame ra file parquet bằng pyarrow
df.to_parquet(PARQUET_DIR / "my_data.parquet", compression="gzip", engine="pyarrow", index=False)

# Đọc lại file parquet
df_loaded = pd.read_parquet(PARQUET_DIR / "my_data.parquet", engine="pyarrow")
print(df_loaded.head())

  CUTOFF_DATE  AGREEMENT_ID DISBURSAL_DATE  DISBURSAL_AMOUNT  DPD_EOM  \
0  2023-01-31       4089415     2023-01-31        18309000.0        0   
1  2023-02-28       4089415     2023-01-31        18309000.0        0   
2  2023-03-31       4089415     2023-01-31        18309000.0        0   
3  2023-04-30       4089415     2023-01-31        18309000.0        0   
4  2023-05-31       4089415     2023-01-31        18309000.0        0   

  RISK_BUCKET  PRINCIPLE_OUTSTANDING STATUS  MOB PRODUCT_TYPE NPA_STAGEID  \
0          B0               18309000      A    0       SALPIL     REGULAR   
1          B0               18273533      A    1       SALPIL     REGULAR   
2          B0               17959922      A    2       SALPIL     REGULAR   
3          B0               17703148      A    3       SALPIL     REGULAR   
4          B0               17414031      A    4       SALPIL     REGULAR   

  DPD_GROUP STATE_MODEL RISK_SCORE  
0      DPD0        DPD0          B  
1      DPD0        DPD0 

In [16]:
pip show pandas pyarrow


Name: pandasNote: you may need to restart the kernel to use updated packages.

Version: 2.3.3
Summary: Powerful data structures for data analysis, time series, and statistics
Home-page: https://pandas.pydata.org
Author: 
Author-email: The Pandas Development Team <pandas-dev@python.org>
License: BSD 3-Clause License
         
         Copyright (c) 2008-2011, AQR Capital Management, LLC, Lambda Foundry, Inc. and PyData Development Team
         All rights reserved.
         
         Copyright (c) 2011-2023, Open source contributors.
         
         Redistribution and use in source and binary forms, with or without
         modification, are permitted provided that the following conditions are met:
         
         * Redistributions of source code must retain the above copyright notice, this
           list of conditions and the following disclaimer.
         
         * Redistributions in binary form must reproduce the above copyright notice,
           this list of conditions and

In [5]:
import pandas as pd

# Giả sử DataFrame của bạn tên là df
df.to_parquet("output.parquet", compression="snappy", index=False)


In [6]:
df = load_data(r"D:\Python_code\RR_Model\sql\ETB.sql")

🔗 Loading data from Oracle...
=== SQL DEBUG ===
File: D:\Python_code\RR_Model\sql\ETB.sql
First 200 chars:
 select  CUTOFF_DATE, AGREEMENT_ID, DISBURSAL_DATE, DISBURSAL_AMOUNT, DPD_EOM, RISK_BUCKET, PRINCIPLE_OUTSTANDING, STATUS, MOB, MAFC_SUB_CATEGORY PRODUCT_TYPE, NPA_STAGEID, DPD_GROUP, STATE_MODEL, CASE...
Params: {}


D:\Python_code\RR_Model\src\db.py:63: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(final_sql, conn, params=bind_params)


In [7]:
import pandas as pd

# Giả sử DataFrame của bạn tên là df
df.to_parquet("ETB.parquet", compression="snappy", index=False)
